# Verifying the Accuracy of iGEM Toronto's Flux Scanning Based on Enforced objective Flux (FSEOF) Implementation

As part of our 2023 project, we (the Dry Lab sub-team of the University of Toronto's iGEM team) made a COBRApy implementation of the FSEOF (Flux Scanning based on Enforced Objective Flux) algorithm [(Choi et al. 2010)](https://pubmed.ncbi.nlm.nih.gov/20348305/). FSEOF is used to identify candidate genes that can be overexpressed in order to achieve a metabolic engineering objective. In [(Choi et al. 2010)](https://pubmed.ncbi.nlm.nih.gov/20348305/), FSEOF was used to identify gene over-expression targets in lycopene-producing *E. coli* for increased production of lycopene.

In order to validate our implementation of FSEOF, we applied it to lycopene-producing *E. coli* modeled in COBRApy, to see if our implementation outputted similar over-expression candidate reactions for improved lycopene biosynthesis as in [(Choi et al. 2010)](https://pubmed.ncbi.nlm.nih.gov/20348305/).

Throughout the rest of this notebook example, we refer to our team implementation of FSEOF as cobra-fseof, and the original FSEOF algorithm in [(Choi et al. 2010)](https://pubmed.ncbi.nlm.nih.gov/20348305/), as just FSEOF.


In [5]:
import sys
sys.path.append("..")

## Construction of the lycopene-producing *E. coli model in COBRApy*

The parental *E. coli* strain used in [(Choi et al. 2010)](https://pubmed.ncbi.nlm.nih.gov/20348305/) to experimentally validate the targets identified by FSEOF is a "recombinant E. coli DH5 alpha strain that contains the *Erwinia uredovora crtEIB* (lycopene biosynthesis) genes and the *E. coli dxs* gene." (Choi et al. 2010)
The genome scale metabolic model (GSMM) used in [(Choi et al. 2010)](https://pubmed.ncbi.nlm.nih.gov/20348305/) for the *in-silico* metabolic modeling (through MetaFluxNet) and FSEOF simulations of the recombinant E. coli DH5 alpha strain, was EcoMBEL979, which was expanded to include the exogenous lycopene biosynthetic pathways and associated genes (Choi et al. 2010).

Instead of using the expanded EcoMBEL979 GSMM for the *in-silico* metabolic modeling and cobra-fseof simulations of the recombinant E. coli DH5 alpha strain, we used the already available BiGG model for E. coli DH5 alpha [iEC1368_DH5a](https://pubmed.ncbi.nlm.nih.gov/20348305/), and expanded it to include the necessary lycopene biosynthetic pathways and the genes *crtE, crtB* and *crtI*, that mediate these pathways. In addition to the lycopene biosynthetic genes, a lycopene demand reaction was added. More information on the exact names of the genes, pathways and metabolites that were added can be found in Choi et al. 2010 supplementary file 3, Table S3A and B.


In [6]:
import pathlib
import cobra
import cobra.manipulation

def ecoli_dh5a(variant: str = "base") -> cobra.Model:


    if variant in {"base", "corrected"}:
        sbml_path = "iEC1368_DH5a.xml"
        return cobra.io.read_sbml_model(str(sbml_path))

    else:
        raise ValueError()

#load base model: E. coli DH5 alpha from BiGG
model = ecoli_dh5a()


In [8]:
from typing import Dict, Optional

import cobra
import cobra.manipulation
from cobra import Reaction


def add_single_gene_reaction_pair_lyc(
    model: cobra.Model,
    gene_id: str,
    reaction_id: str,
    reaction_name: str,
    metabolites: Dict[str, float],
    gene_name: Optional[str] = None,
) -> None:
    assert not model.genes.query(lambda k: k == gene_id, attribute="id")
    assert not model.reactions.query(lambda k: k == reaction_id, attribute="id")

    Reaction = cobra.Reaction
    rxn = Reaction(id=reaction_id)

    if gene_name is None:
        gene_name = gene_id
    gene = cobra.Gene(gene_id, name=gene_name)

    model.add_reactions([rxn])
    model.genes.add(gene)

    rxn.name = reaction_name
    rxn.bounds = (-1000, 1000)  # TODO: set to something? Assumes that reaction is reversible.
    rxn.add_metabolites(metabolites)
    rxn.gene_reaction_rule = gene_id


def add_crtE(model: cobra.Model) -> None:
    # References: Choi et. al 2010 supplementary file 3

    ggpp = cobra.Metabolite(
        id="ggpp",
        formula="C20H33O7P2",
        name="geranylgeranyl diphosphate",
        charge=-3,
        compartment="c",
    )
    model.add_metabolites([ggpp])

    phyto = cobra.Metabolite(
        id="phyto",
        formula="C40H64",
        name="phytoene",
        charge=0,
        compartment="c",
    )
    model.add_metabolites([phyto])

    lyco = cobra.Metabolite(
        id="lyco",
        formula="C40H56",
        name="lycopene",
        charge=0,
        compartment="c",
    )
    model.add_metabolites([lyco])

    add_single_gene_reaction_pair_lyc(
        model=model,
        gene_id="crtE",
        reaction_id="ZCRTE",
        reaction_name="Synthesis of geranylgeranyl pyrophosphate",
        metabolites={
            "ipdp_c": -1.0,
            "frdp_c": -1.0,
            "ggpp": 1.0,
            "ppi_c": 1.0
        },
    )


def add_crtB(model: cobra.Model) -> None:
    # References: Choi et. al 2010 supplementary file 3

    add_single_gene_reaction_pair_lyc(
        model=model,
        gene_id="crtB",
        reaction_id="ZCRTB",
        reaction_name="Synthesis of phytoene",
        metabolites={
            "ggpp": -2.0,
            "phyto": 1.0,
            "ppi_c": 1.0
        },
    )


def add_crtI(model: cobra.Model) -> None:
    # References: Choi et. al 2010 supplementary file 3

    add_single_gene_reaction_pair_lyc(
        model=model,
        gene_id="crtI",
        reaction_id="ZCRTI",
        reaction_name="synthesis of lycopene from phytoene (dehydrogenation rxn)",
        metabolites={
            "phyto": -1.0,
            "fad_c": -8.0,
            "lyco": 1.0,
            "fadh2_c": 8.0
        },
    )

def add_lycopene_demand(model: cobra.Model) -> None:
    # References: Choi et. al 2010 supplementary file 3,

    lyco_demand = Reaction('LYCOdemand')
    model.add_reactions([lyco_demand])
    lyco_demand.name = 'Lycopene demand rxn'
    lyco_demand.lower_bound = 0
    lyco_demand.upper_bound = 1000
    lyco_demand.add_metabolites({"lyco": -1.0})


#add necessary lycopene biosynthesis genes + associated pathways to the base model to
#create the recombinant E. coli DH5 alpha model
add_crtE(model)
add_crtB(model)
add_crtI(model)

#add the lycopene demand reaction
add_lycopene_demand(model)




AssertionError: 

## Media Setup

After the creation of the recombinant E. coli DH5 alpha COBRApy model, the growth media must be simulated in COBRApy. The media used to grow the recombinant E. coli DH5 alpha cells was a [2xYT medium](https://sharebiology.com/2x-yt-medium/)(Choi et al. 2010). Unfortunately, due to the lack of availability of exact nutrients/metabolites (exchange reactions) needed to simulate the 2xYT medium in COBRApy, we decided to simulate a closely related medium called LB medium, which we were able to find all the metabolites, and thus the exchange reactions for, here: [LB medium](https://github.com/cdanielmachado/carveme/blob/master/carveme/data/benchmark/media_db.tsv). The exact flux for all the nutrient exchange reactions was not available, so a flux of 3 mmol / [gDW h] was chosen for each.

In [9]:

def simulate_LB_media(model: cobra.Model) -> None:


    LB_MEDIA_COMP = ["EX_adn_e", "EX_ala__L_e", "EX_amp_e", "EX_arg__L_e",
                     "EX_aso3_e", "EX_asp__L_e", "EX_ca2_e", "EX_cbl1_e",
                     "EX_cd2_e", "EX_cl_e", "EX_cmp_e", "EX_cobalt2_e",
                     "EX_cro4_e", "EX_cu2_e", "EX_cys__L_e", "EX_dad_2_e",
                     "EX_dcyt_e", "EX_fe2_e", "EX_fe3_e", "EX_fol_e", "EX_glc__D_e",
                     "EX_glu__L_e", "EX_gly_e", "EX_gmp_e", "EX_gsn_e", "EX_h2o_e",
                     "EX_h2s_e", "EX_h_e", "EX_hg2_e", "EX_his__L_e", "EX_hxan_e",
                     "EX_ile__L_e", "EX_ins_e", "EX_k_e", "EX_leu__L_e", "EX_lipoate_e",
                     "EX_lys__L_e", "EX_met__L_e", "EX_mg2_e", "EX_mn2_e", "EX_mobd_e",
                     "EX_na1_e", "EX_nac_e", "EX_nh4_e", "EX_ni2_e", "EX_o2_e",
                     "EX_phe__L_e", "EX_pheme_e", "EX_pi_e", "EX_pnto__R_e",
                     "EX_pro__L_e", "EX_pydx_e", "EX_ribflv_e", "EX_ser__L_e",
                     "EX_so4_e", "EX_thm_e", "EX_thr__L_e", "EX_thymd_e", "EX_trp__L_e",
                     "EX_tyr__L_e", "EX_ump_e", "EX_ura_e", "EX_uri_e", "EX_val__L_e",
                     "EX_zn2_e"
                      ]

    for metabolite in LB_MEDIA_COMP:
        model.medium[metabolite] = 3

#set LB media
simulate_LB_media(model)

FSEOF from cobra-fseof with 9 steps, setting lycopene production as enforced objective, biomass as main objective, and enforced direction as max, was performed on the recombinant, lycopene producing E. coli DH5 alpha model from above, to see whether resulting reactions chosen as overexpression candidates match those reported in [(Choi et al. 2010)](https://pubmed.ncbi.nlm.nih.gov/20348305/).

In [10]:
#run cobra-fseof
from cobra_fseof.fseof import fseof
results = fseof(model, 9, "LYCOdemand", "BIOMASS_Ec_iJO1366_core_53p95M", "max")
#print(results.scan)

FSEOF; Running FVA: 100%|██████████| 9/9 [05:47<00:00, 38.57s/it]


Here, we have printed the list of reactions that cobra-fseof identiified as targets for overexpression for increased lycopene production.

In [12]:
targets = zip(results.scan.index, results.scan.target)
targets = dict(targets)
lst = []
for target in targets:
    if targets[target]:
        lst.append(target)


import pandas as pd
df = pd.DataFrame(lst)
df

,0
0,AKGDH
1,CDPMEK
2,CYTK1
3,DMATT
4,DXPRIi
5,DXPS
6,FBA3
7,FE3Ri
8,FESD1s
9,FESR


Here, we have listed out and categorized (by which main pathways they belong to) all target over-expression reactions for increased lycopene production identified by FSEOF. This data was taken from Choi et al. 2010 Supplemental Table 4A. We first examined this table and identified the corresponding reactions in our COBRApy model, however some exceptions were made:

* Of the reactions identified from the TCA cycle, 'FUM_rxn' was not found in the COBRApy model, so it is replaced with the reaction 'FUM'.

* Of the reactions identified from the lycopene biosynthetic pathway, 'MECHPDH' was not found in the COBRApy model. However, 'MECDPDH2' and 'MECDPDH5' which correspond to the same reaction were found and are listed here.



In [13]:

#TCA cycle reactions
tableFourTCA = ['ACONT', 'CS', 'FUM', 'ICDHyr', 'MDH', 'SUCDli', 'SUCOAS', 'AKGDH', 'SUCD4']


#lycopene biosynthetic pathway
tableFourLyc = ['CDPMEK', 'DMATT', 'DXPRIi', 'DXPS','GRTT', 'IPDDI', 'IPDPS', 'MECDPDH5', 'MECDPDH2', 'MECDPS', 'MEPCT', 'ZCRTE', 'ZCRTB', 'ZCRTI']

#Glycolysis
tableFourGlyc = ['FBA', 'PFK', 'PGI', 'TPI']


#disclaimer: 'CACTP' is replaced by 'CRNabcpp' (since model did not have CACTP rxn)
#CAITP is replaced by CRNt8pp (since model did not have CAITP rxn)
tableFourOther = ['ADK1', 'ADK4', 'CYTK1', 'CRNabcpp', 'CRNt8pp', 'PPA', 'CO2t', 'H2Ot', 'PIt2rpp']




In [9]:
#Of the genes in supp file 4, how many was our algorithm able to identify?

tableFourMegalst = tableFourTCA + tableFourLyc + tableFourGlyc + tableFourOther
print("Total number of reactions that the choi et. al 2010 paper identified: " + str(len(tableFourMegalst)))

count = 0
for item in tableFourMegalst:
    if item in lst:
        count += 1

print("How many out of the total number of reactions that the choi et. al paper identified, did our FSEOF identify: " + str(count) + "/" + str(len(tableFourMegalst)))
print("Above value as a percentage: " + str(count/len(tableFourMegalst)))


count1 = 0
for item in tableFourLyc:
    if item in lst:
        count1 += 1

print("What percentage of the Lycopene biosynthetic pathways that choi et. al identified, did our fseof identify: " + str(count1/len(tableFourLyc)))




Total number of reactions that the choi et. al 2010 paper identified: 36
How many out of the total number of reactions that the choi et. al paper identified, did our FSEOF identify: 21/36
Above value as a percentage: 0.5833333333333334
What percentage of the Lycopene biosynthetic pathways that choi et. al identified, did our fseof identify: 1.0
